In [2]:
import bw2data as bd
import bw2calc as bc
import bw2io as bi
import os               # to use "operating system dependent functionality"
import numpy as np      # "the fundamental package for scientific computing with Python"
import pandas as pd     # "high-performance, easy-to-use data structures and data analysis tools" for Python

In [3]:
bd.projects.set_current("ei38-teaching-25")

In [4]:
bd.databases

Databases dictionary with 6 object(s):
	EXIOBASE 3.8.1 2017 monetary
	EXIOBASE 3.8.1 2017 monetary biosphere
	Korean Motors Chaebol
	biosphere3
	ei 3.8 cutoff
	강한 국가를 위한 영광스러운 구리

In [93]:
if "charcrete" in bd.databases:
    del bd.databases["charcrete"]

In [3]:
bd.projects.set_current("AS-group-project")

In [4]:
bd.databases

Databases dictionary with 5 object(s):
	EXIOBASE 3.8.1 2017 monetary
	EXIOBASE 3.8.1 2017 monetary biosphere
	biosphere3
	charcrete_db
	ei 3.8 cutoff

In [7]:
eidb = bd.Database("ei 3.8 cutoff")

In [33]:
forwarding_act = [act for act in eidb if 'forwarding' in act['name'] and act['location']=="RER"][0]

In [114]:
forwarding_act.as_dict()

{'comment': 'The inventory refers to 1 productive machine hour (PMS) - referring to the operational time of the machine including breaks up to 15 minutes - of a forwarder class II (10 - 12 t) with an average engine power of 110 kW and a total weight of 11,049 kg.\nA forwarder is a forestry vehicle that carries felled logs from the stump to a roadside landing. Unlike a skidder, a forwarder carries logs clear of the ground, which can reduce soil impacts but tends to limit the size of the logs it can move. Forwarders are typically employed together with harvesters in cut-to-length logging operations.\nImage: https://db3.ecoinvent.org/images/8840519f-6554-4a44-8589-fe5c5b02e11c\nGeography:  Emission profile according to European legislation\nTechnology:  Average european technology',
 'classifications': [('ISIC rev.4 ecoinvent', '0220:Logging'),
  ('CPC', '8614: Support and operation services to forestry and logging')],
 'activity type': 'ordinary transforming activity',
 'activity': '44df

In [40]:
chipping_act = [act for act in eidb if 'chipping' in act['name'] and 'mobile' in act['name'] and act['location']=="RER"][0]

In [116]:
chipping_act['reference product']

'wood chipping, chipper, mobile, diesel, at forest road'

In [52]:
chip_transport = [
act for act in eidb if 
     "market for transport, freight, lorry, unspecified" in act['name'] 
    and act['location']=='RER'
    and not "generic" in act['name']
][0]

In [117]:
chip_transport.as_dict()

{'comment': "This is the market for 'transport, freight, lorry, unspecified', in the the geography of Europe. This is a generic market; the suppliers of this market make available this generic product by redirecting a part of the production of several more specific products.\nThis is a market activity. Each market represents the consumption mix of a product in a given geography, connecting suppliers with consumers of the same product in the same geographical area. Markets group the producers and also the imports of the product (if relevant) within the same geographical area. They also account for transport to the consumer and for the losses during that process, when relevant.\nNo transport or losses have been considered for the generic product supplied in this market, as the transport modelled in the upstream niche markets for the more specific products do already account it.\nThis is delivering the service of transportation of 1 metric ton across the distance of 1 km. This service onl

In [58]:
elec = [
act for act in eidb if 
     "market for electricity, high voltage" in act['name'] 
    and act['location']=='SE'
][0]

In [118]:
elec

'market for electricity, high voltage' (kilowatt hour, SE, None)

In [70]:
N20_act = [
act for act in bd.Database('biosphere3') if 
     'Dinitrogen' in act['name'] 
][0]

In [119]:
N20_act

'Dinitrogen monoxide' (kilogram, None, ('air', 'non-urban air or from high stacks'))

In [81]:
CH4_act = [
act for act in bd.Database('biosphere3') if 
     'Methane, non-fossil' in act['name'] 
    and 'high stacks' in str(act['categories'])
][0]

In [121]:
CH4_act.as_dict()

{'categories': ('air', 'non-urban air or from high stacks'),
 'code': '57bdb443-d4a6-423d-8024-959b8261d02e',
 'CAS number': '000074-82-8',
 'name': 'Methane, non-fossil',
 'database': 'biosphere3',
 'unit': 'kilogram',
 'type': 'emission',
 'id': 1690}

In [95]:
db = bd.Database("charcrete_db")

In [96]:
charcrete_activities = {
    ("charcrete_db", '1'): {
        "name": "woodchip production",
        'unit': 'ton',
        'categories': ("comment", "category here"), #?
        'database': 'charcrete_db',
        "exchanges": [{
            "amount": 0.15739,
            "input": ('ei 3.8 cutoff', '39f73b6fa228fc6df946cc78891f1456'), #Forwarding
            "type": "technosphere",
            "uncertainty type":0,
            "unit=": "hour"},
                      {
            "amount": 0.047136,
            "input": ('ei 3.8 cutoff', 'a4f64071a3a353f807b933a6ddfc1f81'), #Chipping
            "type": "technosphere",
            "uncertainty type":0,
            "unit=": "hour"}
        ]
        },
        ("charcrete_db", '2'): {
        "name": "woodchip transport",
        'unit': 'ton',
        'categories': ("comment", "lorry, unspecified, RER - 200km"), #
        'database': 'charcrete_db',
        "exchanges": [{
            "amount": 200,
            "input": ('ei 3.8 cutoff', '2b0c1793924e1bb3e821cbe15bee1a32'), #transport
            "type": "technosphere",
            "uncertainty type":0,
            "unit=": "ton.km"},
        ]
        },
        ("charcrete_db", '3'): {
        "name": "pyrolysis",
        'unit': 'ton',
        'categories': ("comment", "For one ton of biomass, Sweden mix - HV"), #
        'database': 'charcrete_db',
        "exchanges": [{
            "amount": 0.010,
            "input": ('ei 3.8 cutoff', 'b3e6b1396a7aad9719da6e6e4aef374a'), #electricity HV
            "type": "technosphere",
            "uncertainty type":0,
            "unit=": "kWh"},
            {
            "amount": 0.108,
            "input": ('biosphere3', 'afd6d670-bbb0-4625-9730-04088a5b035e'), #N20
            "type": "biosphere",
            "uncertainty type":0,
            "unit=": "kilogram"},
                        {
            "amount": 0.198,
            "input": ('biosphere3', '57bdb443-d4a6-423d-8024-959b8261d02e'), #CH4
            "type": "biosphere",
            "uncertainty type":0,
            "unit=": "kilogram"},
        ]
        },
     ("charcrete_db", '4'): {
        "name": "biochar production",
        'unit': 'ton',
        'categories': ("comment", "category here"),
        'database': 'charcrete_db',
        "exchanges": [{
            "amount": 2.7, #amount is 1/0.37 as we get 370 kilo of biochar out of 1 ton woodchips
            "input": ("charcrete_db", "1"), #woodchip production
            "type": "technosphere",
            "uncertainty type":0,
            "unit": "ton"},
            {
            "amount": 2.7,
            "input": ("charcrete_db", "2"), #woodchip transport
            "type": "technosphere",
            "uncertainty type":0,
            "unit": "ton"},
            {
            "amount": 2.7,
            "input": ("charcrete_db", "3"), #pyrolysis
            "type": "technosphere",
            "uncertainty type":0,
            "unit": "ton"},
                      ]
        }
}

In [97]:
db.write(charcrete_activities)

Not able to determine geocollections for all datasets. This database is not ready for regionalization.
Title: Writing activities to SQLite3 database:
  Started: 10/26/2022 13:41:50
  Finished: 10/26/2022 13:41:50
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 0.37


In [100]:
act_t = db.random()
act_t

'pyrolysis' (ton, None, ('comment', 'For one ton of biomass, Sweden mix - HV'))

In [101]:
ipcc2013 = ('IPCC 2013', 'climate change', 'GWP 100a')

In [104]:
lca_w = bc.LCA({act_t:1},ipcc2013)

In [105]:
lca_w.lci()
lca_w.lcia()
lca_w.score

34.24181453257221

In [106]:
dp = db.datapackage()

In [107]:
dp.metadata

{'profile': 'data-package',
 'name': 'charcrete_db',
 'id': '50c7c6ebaa05458aa475dc00b5fb1472',
 'licenses': [{'name': 'ODC-PDDL-1.0',
   'path': 'http://opendatacommons.org/licenses/pddl/',
   'title': 'Open Data Commons Public Domain Dedication and License v1.0'}],
 'resources': [{'profile': 'data-resource',
   'format': 'npy',
   'mediatype': 'application/octet-stream',
   'name': 'charcrete_db_inventory_geomapping_matrix.indices',
   'matrix': 'inv_geomapping_matrix',
   'kind': 'indices',
   'path': 'charcrete_db_inventory_geomapping_matrix.indices.npy',
   'group': 'charcrete_db_inventory_geomapping_matrix',
   'nrows': 4,
   'category': 'vector'},
  {'profile': 'data-resource',
   'format': 'npy',
   'mediatype': 'application/octet-stream',
   'name': 'charcrete_db_inventory_geomapping_matrix.data',
   'matrix': 'inv_geomapping_matrix',
   'kind': 'data',
   'path': 'charcrete_db_inventory_geomapping_matrix.data.npy',
   'group': 'charcrete_db_inventory_geomapping_matrix',
   'n

In [ ]:
db.new_activity

In [8]:
concrete_20MPa_PE = [
    act for act in eidb if 
    'concrete' in act['name'] 
    and '20MPa' in act['name']
    and 'self' in act['name']
    and 'Portland' in act['name']
    and act['location']=='PE'
][0]

In [9]:
concrete_20MPa_PE.technosphere

<bound method Activity.technosphere of 'concrete production, 20MPa, self-construction, with Portland cement' (cubic meter, PE, None)>

In [10]:
for exc in concrete_20MPa_PE.technosphere() : 
    print(exc['name'], exc['amount'])

gravel, crushed 1063.578
electricity, medium voltage 0.85799
wastewater from concrete production -0.02472
sand 709.052
waste concrete -70.79325
synthetic rubber 0.0345
scrap steel -0.0238
lubricating oil 0.0499
waste mineral oil -0.0499
tap water 325.76325
waste rubber, unspecified -0.0345
cement, Portland 393.975
steel, low-alloyed, hot rolled 0.0238


In [14]:
[act for act in eidb if 'market for transport, freight, lorry, unspecified' in act['name'] and act['location']=='RER' and not 'EURO' in act['name']
][0]['reference product']

'transport, freight, lorry, unspecified'